<a href="https://colab.research.google.com/github/jvrscak/Analiza-utjecaja-prehrane-na-rezultate-sporta-a/blob/main/projekt_iz_PZAP_JanVr%C5%A1%C4%8Dak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uvođenje i učitavanje podataka.

In [1]:
import pandas as pd
import numpy as np
import json